Import OpenAI

In [95]:
from dotenv import load_dotenv
import os
from openai import OpenAI
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)
# Initialize the OpenAI client with the API key

Step 1: Create an Assistant

In [103]:
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math problems.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-3.5-turbo-1106"
)

Step 2: Create a Thread

In [104]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_zprQkHIyZYHZ4gSApEla2dWB', created_at=1720048252, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


Step 3: Add a message to a Thread

In [105]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Solve this problem: 3x + 11 = 14"
)

Step 4: Run the Assistant

In [106]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

Step 5: Display the Assistant's Response

In [107]:
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)

In [108]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

In [109]:
for message in reversed(messages.data):
    if message.role == "assistant":
        for content in message.content:
            print(message.role + ": " + content.text.value)
    else:
        user_content = " ".join([part.text.value for part in message.content])
        print(message.role + ": " + user_content)


user: Solve this problem: 3x + 11 = 14
assistant: The solution to the equation 3x + 11 = 14 is \( x = 1 \).


AI Assistant with Files

Create a Vector Store

In [ ]:
vector_store = client.beta.vector_stores.create(name="memgpt-paper.pdf")
file_paths = ["memgpt-paper.pdf"]
file_streams = [open(path, "rb") for path in file_paths]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

print(file_batch.status)
print(file_batch.file_counts)

Create the Assistant

In [ ]:
assistant = client.beta.assistants.create(
    name="Machine learning researcher",
    instructions="You are a machine learning researcher. Answer questions on the research paper.",
    tools=[{"type": "file_search"}],
    model="gpt-3.5-turbo-1106",
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

Create a Thread and Attach the File

In [ ]:
# Upload the user provided file to OpenAI
message_file = client.files.create(
  file=open("memgpt-paper.pdf", "rb"), purpose="assistants"
)

# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "How does memgpt allow LLMs to have unlimited context length?",
      # Attach the new file to the message
      "attachments": [
        {"file_id": message_file.id, "tools": [{"type": "file_search"}]}
      ],
    }
  ]
)

print(thread.tool_resources.file_search)

Run the Assistant

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

Retrieve and display the Assistant's response

In [ ]:
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)

In [ ]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

In [ ]:
for message in reversed(messages.data):
    if message.content:  # Check if message.content is not empty
        content_str = " ".join([content.text.value for content in message.content])
        print(message.role + ": " + content_str)
    else:
        print(message.role + ": No content")